In [102]:
#  notebook imports

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pickle
import joblib

import pandas as pd
import numpy as np

In [12]:
# gather data

boston_data = load_boston()
data = pd.DataFrame(data=boston_data.data, columns=boston_data.feature_names)
features = data.drop(['INDUS','AGE'],axis=1)

log_prices = np.log(boston_data.target)
target = pd.DataFrame(log_prices, columns=['PRICE']) # converting into 2-d from flat array


In [14]:
features

,CRIM,ZN,CHAS,NOX,RM,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,0.0,0.538,6.575,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,0.0,0.469,6.421,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,0.0,0.469,7.185,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,0.0,0.458,6.998,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,0.0,0.458,7.147,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,0.0,0.573,6.593,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,0.0,0.573,6.120,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,0.0,0.573,6.976,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,0.0,0.573,6.794,2.3889,1.0,273.0,21.0,393.45,6.48


In [24]:
CRIM_IDX =0
ZN_IDX=1
CHAS_IDX =2
RM_IDX=4
PTRATIO_IDX=8

property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIM_IDX]= features['CRIM'].mean()
# property_stats[0][ZN_IDX]= features['ZN'].mean()
# property_stats[0][CHAS_IDX]= features['CHAS'].mean()

property_stats = features.mean().values.reshape(1,11)

In [25]:
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [28]:
regr = LinearRegression().fit(features,target)
fitted_vals = regr.predict(features)

# calcualating mse and rmse
mse_values = mean_squared_error(target, fitted_vals)
rmse_values = np.sqrt(mse_values)

In [30]:
rmse_values

0.18751213519713034

In [69]:
# define a function to output the log_price

def get_log_estimate(nr_rooms,
                    students_per_class,
                    next_to_river=False,
                    high_confidence=True):
    # configure property
    property_stats[0][RM_IDX]= nr_rooms
    property_stats[0][PTRATIO_IDX]= students_per_class
    if next_to_river:
        property_stats[0][CHAS_IDX]= 1
    else:
        property_stats[0][CHAS_IDX]= 0

     # make predictions
    log_estimate = regr.predict(property_stats)[0][0]
    
    # calculate range
    
    if high_confidence:
        # for 2 stdev
        upper_bound = log_estimate+2*rmse_values
        lower_bound = log_estimate-2*rmse_values
        interval = 95
    
    else:
        # for 1 stdev
        upper_bound = log_estimate+rmse_values
        lower_bound = log_estimate-rmse_values
        interval= 68
    
#     actual_price = np.e**log_estimate+25.56*(np.e**log_estimate)
        
    return log_estimate, upper_bound, lower_bound, interval
    

In [64]:
get_log_estimate(3,20,False,False)

(384.14345658427897,
 2.6716097430689403,
 2.8591218782660706,
 2.48409760787181,
 68)

In [66]:
# pull out median price from the dataset

original_median_price = np.median(boston_data.target)

In [81]:
ZILLOW_MEDAIN_PRICE = 583.3
scale_factor = ZILLOW_MEDAIN_PRICE/original_median_price
log_est, u_bound, l_bound, interval = get_log_estimate(9, students_per_class=15, next_to_river=False,
                                                      high_confidence=False)

# convert to today's price (scaling and rounding)

dollar_est = round(np.e**log_est*1000*scale_factor,-3)
u_bound = round(np.e**u_bound*1000*scale_factor,-3)
l_bound = round(np.e**l_bound*1000*scale_factor,-3)
dollar_est

print(f'the estimated dollar value is: {dollar_est}')
print(f'At {interval}% confidence interval the valuation range for this property is: USD {l_bound} to USD {u_bound}')


the estimated dollar value is: 827000.0
At 68% confidence interval the valuation range for this property is: USD 685000.0 to USD 997000.0


In [96]:
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """
    Estimate the price of property in Boston.
    
    parameters:
    --------------------------------------------
    
    rm: number of rooms for the property
    ptratio: student to teacher ratio in classroom
    chas: is it next to Charles River
    large_range: if you want 95% confidence interval for your prediction else gives 68%
    """
    
    if rm < 1 or ptratio < 1 :
        print('That is unrealistic. Try again!')
        return 
    
    
    log_est, u_bound, l_bound, interval = get_log_estimate(rm, students_per_class=ptratio, next_to_river=chas,
                                                      high_confidence=large_range)

    # convert to today's price (scaling and rounding)

    dollar_est = round(np.e**log_est*1000*scale_factor,-3)
    u_bound = round(np.e**u_bound*1000*scale_factor,-3)
    l_bound = round(np.e**l_bound*1000*scale_factor,-3)
    
    return dollar_est, u_bound,l_bound

#     print(f'the estimated dollar value is: {dollar_est}')
#     print(f'At {interval}% confidence interval the valuation range for this property is: USD {l_bound} to USD {u_bound}')


In [97]:
get_dollar_estimate(rm=0, ptratio=200, chas=True)

That is unrealistic. Try again!


In [98]:
import Bostonvaluation as val

In [99]:
val.get_dollar_estimate(6,12,True)

the estimated dollar value is: 783000.0
At 95% confidence interval the valuation range for this property is: USD 538000.0 to USD 1139000.0


In [104]:
joblib.dump(regr, "boston_valuation.pkl")

['boston_valuation.pkl']